# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 7. Support Vector Machine (SVM) Lanjutan


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [41]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-05-01 13:04:51--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.2’

iris.csv.2          100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-05-01 13:04:51 (48.9 MB/s) - ‘iris.csv.2’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [42]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [43]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [44]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [45]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


Pisahkan label/kelas dari data uji menjadi sebuah variabel bernama **label_uji**

In [46]:
label_uji = data_uji.pop('variety')

## 3) Pembentukan data latih one-vs-rest

Metode one-vs-rest memerlukan tiga jenis data latih yang diperlukan untuk melatih tiga SVM yang berbeda pada dataset Iris. Fungsi **buat_trainingset** digunakan untuk membentuk tiga dataset tersebut.

In [47]:
def buat_trainingset(dataset):
  trainingset = {}
  kolom_kelas = dataset.columns[-1]
  list_kelas = dataset[kolom_kelas].unique()
  for kelas in list_kelas:
    data_temp = dataset.copy(deep=True)
    data_temp[kolom_kelas] = data_temp[kolom_kelas].map({kelas:1})
    data_temp[kolom_kelas] = data_temp[kolom_kelas].fillna(-1)
    trainingset[kelas] = data_temp
  return trainingset

In [48]:
kolom_kelas = data.columns[-1]
list_kelas = data[kolom_kelas].unique()
print(list_kelas)
for kelas in list_kelas:
  print(data['sepal.width'])

['Setosa' 'Versicolor' 'Virginica']
0      3.5
1      3.0
2      3.2
3      3.1
4      3.6
      ... 
145    3.0
146    2.5
147    3.0
148    3.4
149    3.0
Name: sepal.width, Length: 150, dtype: float64
0      3.5
1      3.0
2      3.2
3      3.1
4      3.6
      ... 
145    3.0
146    2.5
147    3.0
148    3.4
149    3.0
Name: sepal.width, Length: 150, dtype: float64
0      3.5
1      3.0
2      3.2
3      3.1
4      3.6
      ... 
145    3.0
146    2.5
147    3.0
148    3.4
149    3.0
Name: sepal.width, Length: 150, dtype: float64


Gunakan fungsi **buat_trainingset** untuk membentuk data latih dengan nama variabel **trainingset** yang akan digunakan pada proses training.

In [49]:
trainingset = buat_trainingset(data_latih)

Tampilkan isi **trainingset** agar Anda dapat memahami struktur dari variabel tersebut.

In [50]:
trainingset

{'Versicolor':      sepal.length  sepal.width  petal.length  petal.width  variety
 86            6.7          3.1           4.7          1.5      1.0
 9             4.9          3.1           1.5          0.1     -1.0
 55            5.7          2.8           4.5          1.3      1.0
 148           6.2          3.4           5.4          2.3     -1.0
 61            5.9          3.0           4.2          1.5      1.0
 ..            ...          ...           ...          ...      ...
 108           6.7          2.5           5.8          1.8     -1.0
 135           7.7          3.0           6.1          2.3     -1.0
 83            6.0          2.7           5.1          1.6      1.0
 25            5.0          3.0           1.6          0.2     -1.0
 27            5.2          3.5           1.5          0.2     -1.0
 
 [120 rows x 5 columns],
 'Setosa':      sepal.length  sepal.width  petal.length  petal.width  variety
 86            6.7          3.1           4.7          1.5     -1

## 4) Pembentukan SVM Biner

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [51]:
def hitung_cost_gradient(W, X, Y, regularization):
  jarak = 1 - (Y * np.dot(X, W))
  dw = np.zeros(len(W))
  if max(0, jarak) == 0:
    di = W
  else: di = W - (regularization * Y * X)
  dw += di
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD

In [52]:
from sklearn.utils import shuffle
def sgd(data_latih, label_latih, learning_rate=0.000001, max_epoch=1000, regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1, max_epoch):
    X, Y = shuffle(data_latih, label_latih, random_state=101)
    for index, x in enumerate(X):
      delta = hitung_cost_gradient(bobot, x, Y[index], regularization)
      bobot = bobot - (learning_rate * delta)
  return bobot

## 5) Proses Training

Proses training dilakukan dengan memanggil fungsi **sgd** berulang kali sesuai banyaknya kelas yang ada pada data. Dengan demikian, proses training menghasilkan bobot sebanyak kelas yang ada pada dataset. Buatlah fungsi bernama **training** yang digunakan untuk melakukan proses training one-vs-rest

In [53]:
def training(trainingset):
  list_kelas = trainingset.keys()
  w = {}
  for kelas in list_kelas:
    data_latih = trainingset[kelas]
    label_latih = data_latih.pop(data_latih.columns[-1])
    w[kelas] = sgd(data_latih, label_latih)
  return w

Lakukan proses training dengan memanggil fungsi **training** dan menempatkan hasilnya pada variabel **W**

In [54]:
W = training(trainingset)

Tampilkan isi variabel **W**

In [55]:
W

{'Versicolor': array([ 0.61148307, -1.34648666,  1.35242873, -3.15037139]),
 'Setosa': array([ 0.17160236,  0.58131521, -0.93216226, -0.40457947]),
 'Virginica': array([-3.41772045, -3.42369865,  3.60332579,  7.43085839])}

## 6) Proses *testing* biner
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [56]:
def testing(W, data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W, data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi, y_prediksi)
  return prediksi

In [57]:
label_uji

31         Setosa
116     Virginica
119     Virginica
18         Setosa
104     Virginica
24         Setosa
22         Setosa
8          Setosa
47         Setosa
34         Setosa
140     Virginica
124     Virginica
94     Versicolor
3          Setosa
93     Versicolor
117     Virginica
76     Versicolor
149     Virginica
98     Versicolor
112     Virginica
19         Setosa
67     Versicolor
11         Setosa
129     Virginica
41         Setosa
52     Versicolor
49         Setosa
29         Setosa
97     Versicolor
75     Versicolor
Name: variety, dtype: object

## TUGAS
Pada tugas kali ini Anda mendefinisikan proses testing pada metode one-vs-rest. Proses testing pada metode one-vs-rest dilakukan dengan memanggil proses testing biner untuk setiap **value** pada dictionary **W**. Kelas pada sebuah data latih adalah **key** pada dictionary **W** yang memiliki nilai prediksi **1**. Lengkapi fungsi **testing_onevsrest** di bawah ini. Output dari fungsi tersebut adalah list nama kelas hasil prediksi.

In [58]:
def testing_onevsrest(W,data_uji):
  #tulis kode Anda di sini.
  kelas_prediksi = [""] * len(data_uji)
  for i in range(data_uji.shape[0]):
    for key, value in W.items():
      y_prediction= np.sign(np.dot(value,data_uji.to_numpy()[i]))
      if y_prediction==1:
        kelas_prediksi[i]=key
  return kelas_prediksi

In [59]:
prediksi = testing_onevsrest(W,data_uji)

Berapa banyak data latih yang berhasil diprediksi dengan benar?

In [60]:
prediksi_benar = sum(prediksi == label_uji)

In [61]:
print(prediksi_benar)

29


Praktikum ini melakukan pengujian pada setiap titik data menggunakan metode klasifikasi one-vs-rest, di mana prediksi dilakukan secara terpisah untuk setiap kelas. Kelas yang diprediksi adalah kelas di mana prediksi model sama dengan 1. Fungsi testing_onevsrest didefinisikan dan dieksekusi untuk memprediksi kelas-kelas dari data uji.

Kita melakukan perhitungan jumlah prediksi yang benar (yakni, di mana label yang diprediksi cocok dengan label sesungguhnya). Sebanyak 29 dari data uji berhasil diprediksi dengan benar oleh model.

:)